## Extended Maximum Likelihood fit

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from iminuit import Minuit
from iminuit.cost import ExtendedUnbinnedNLL
from distributions_nbs import g_s_nb, h_s_nb, g_b_nb, h_b_nb, g_s_nb_, h_s_nb_, g_b_nb_, h_b_nb_
from distributions import *
from numba import njit, float64


In [45]:
#Load the data
data = np.loadtxt('data_storage/joint_pdf_sample.csv', delimiter=',', skiprows=1)
#data = np.vstack((x_samples, y_samples)).T
#N = len(x_samples)
X, Y = data.T
N = len(X)

In [48]:
#Create the model
def model(X, Y, mu, sigma, beta, m, f, lmbda, mu_b, sigma_b):
    signal = f * g_s(X, beta, m, mu, sigma, 5.0, 0.0) * h_s(Y, 10.0, 0.0, lmbda)
    background = (1-f) * g_b(X, 5.0, 0.0) * h_b(Y, mu_b, sigma_b, 10.0, 0.0)
    return signal + background
    
def density(data, N, mu, sigma, beta, m, f, lmbda, mu_b, sigma_b):
    X, Y = data
    return N, N * model (X, Y, mu, sigma, beta, m, f, lmbda, mu_b, sigma_b)

In [49]:
#make the cost function which is the negative log likelihood
nll = ExtendedUnbinnedNLL((X,Y), density)

#Define the true parameters as starting values
N_ = 100000
mu_ = 3.0
sigma_ = 0.3
beta_ = 1.0
m_ = 1.4
f_ = 0.6
lmbda_ = 0.3
mu_b_ = 0.0
sigma_b_ = 2.5

mi = Minuit(nll, 
            N=N_,          # name=value format
            mu=mu_,
            sigma=sigma_,
            beta=beta_,
            m=m_,
            f=f_,
            lmbda=lmbda_,
            mu_b=mu_b_,
            sigma_b=sigma_b_)

mi.limits["N"] = (0, None)
mi.limits["mu"] = (0, 5)
mi.limits["sigma"] = (0, 3)
mi.limits["beta"] = (0, 3)
mi.limits["m"] = (0, 2)
mi.limits["lmbda"] = (0, 1)
mi.limits["mu_b"] = (-5, 5)
mi.limits["sigma_b"] = (0, 5)
mi.limits["f"] = (0, 1)

mi.migrad()
mi.hesse()


┌─────────────────────────────────────────────────────────────────────────┐
│                                Migrad                                   │
├──────────────────────────────────┬──────────────────────────────────────┤
│ FCN = -1.62e+06                  │              Nfcn = 421              │
│ EDM = 0.000995 (Goal: 0.0002)    │                                      │
├──────────────────────────────────┼──────────────────────────────────────┤
│          Valid Minimum           │   Below EDM threshold (goal x 10)    │
├──────────────────────────────────┼──────────────────────────────────────┤
│     SOME parameters at limit     │           Below call limit           │
├──────────────────────────────────┼──────────────────────────────────────┤
│             Hesse ok             │     Covariance FORCED pos. def.      │
└──────────────────────────────────┴──────────────────────────────────────┘
┌───┬─────────┬───────────┬───────────┬────────────┬────────────┬─────────┬─────────┬───────┐
│   │ Name    │   Value   │ Hesse Err │ Minos Err- │ Minos Err+ │ Limit-  │ Limit+  │ Fixed │
├───┼─────────┼───────────┼───────────┼────────────┼────────────┼─────────┼─────────┼───────┤
│ 0 │ N       │ 100.000e3 │  0.001e3  │            │            │    0    │         │       │
│ 1 │ mu      │  2.9772   │  0.0004   │            │            │    0    │    5    │       │
│ 2 │ sigma   │ 258.71e-3 │  0.35e-3  │            │            │    0    │    3    │       │
│ 3 │ beta    │  1.5808   │  0.0033   │            │            │    0    │    3    │       │
│ 4 │ m       │  2.00000  │  0.00027  │            │            │    0    │    2    │       │
│ 5 │ f       │  1.0000   │  0.0016   │            │            │    0    │    1    │       │
│ 6 │ lmbda   │ 230.0e-3  │  0.6e-3   │            │            │    0    │    1    │       │
│ 7 │ mu_b    │     0     │     6     │            │            │   -5    │    5    │       │
│ 8 │ sigma_b │    5.0    │    3.3    │            │            │    0    │    5    │       │
└───┴─────────┴───────────┴───────────┴────────────┴────────────┴─────────┴─────────┴───────┘
┌─────────┬─────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┐
│         │               N              mu           sigma            beta               m               f           lmbda            mu_b         sigma_b │
├─────────┼─────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┤
│       N │           0.935               0               0               0               0        -0.7e-12              -0             0.0          -0.000 │
│      mu │               0        1.97e-07        -0.01e-6        -0.11e-6         0.5e-12        -0.1e-12               0        12.28e-6        -0.01e-6 │
│   sigma │               0        -0.01e-6        1.22e-07         0.12e-6        -0.7e-12         3.3e-12              -0      -295.91e-6         0.20e-6 │
│    beta │               0        -0.11e-6         0.12e-6         1.1e-05        37.8e-12      -101.7e-12               0        9.079e-3       -0.006e-3 │
│       m │               0         0.5e-12        -0.7e-12        37.8e-12        3.13e-12               0              -0    -152.3305e-9       103.3e-12 │
│       f │        -0.7e-12        -0.1e-12         3.3e-12      -101.7e-12               0        1.19e-12        -7.6e-12 -105.9154819e-6      71.8281e-9 │
│   lmbda │              -0               0              -0               0              -0        -7.6e-12        3.59e-07        675.7e-6         -0.5e-6 │
│    mu_b │             0.0        12.28e-6      -295.91e-6        9.079e-3    -152.3305e-9 -105.9154819e-6        675.7e-6        9.48e+03          -6.414 │
│ sigma_b │          -0.000        -0.01e-6         0.20e-6       -0.006e-3       103.3e-12      71.8281e-9         -0.5e-6     